Ce fichier est une première version ''essai'' pour merge des entités et non des triplets, extraits de mRebel.
Il n'a pas besoin d'amélioration, c'est une ancienne version, la nouvelle favorise le merge sur toute la relation.

Using ALl-MiniLM to merge entites from a triplets extration LLM

In [ ]:
%%capture
!pip install sentence-transformers torch

In [ ]:
%%capture
from sentence_transformers import SentenceTransformer, util
import torch

# Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Examples of relations

In [ ]:
r1 = {'head': 'Université Lyon 2', 'head_type': 'org', 'type': 'subsidiary', 'tail': 'Paris School of Economics', 'tail_type': 'org'}
r2 = {'head': 'Université Lumière Lyon 2', 'head_type': 'org', 'type': 'part of', 'tail': 'GATE', 'tail_type': 'org'}

relations = [r1,r2]

Comparing loop

In [ ]:
def merge_entities(relations, threshold = 0.7):
  for i in range(len(relations)):
      for j in range(i+1, len(relations)):
          if relations[i]['head_type'] == relations[j]['head_type']:
              entity1 = relations[i]['head']
              entity2 = relations[j]['head']

              embeddings1 = model.encode(entity1, convert_to_tensor=True)
              embeddings2 = model.encode(entity2, convert_to_tensor=True)

              similarity = util.pytorch_cos_sim(embeddings1, embeddings2).item()
              print("Similarity: ", similarity)
              if similarity >= threshold:
                  # Choose which entity to keep (e.g., the more specific one)
                  chosen_entity = entity1 if len(entity1) < len(entity2) else entity2
                  # Update the head of the second relation
                  relations[j]['head'] = chosen_entity

Pretty print

In [ ]:
def pretty_print_relations(relations):
    for relation in relations:
        print("Relation:")
        for key, value in relation.items():
            print(f"    {key}: {value}")

In [ ]:
merge_entities(relations)
pretty_print_relations(relations)

Similarity:  0.9405850172042847
Relation:
    head: Université Lyon 2
    head_type: org
    type: subsidiary
    tail: Paris School of Economics
    tail_type: org
Relation:
    head: Université Lyon 2
    head_type: org
    type: part of
    tail: GATE
    tail_type: org


Autre test


In [ ]:
r3 = {'head': 'Napoleon', 'head_type': 'per', 'type': 'applies to jurisdiction', 'tail': 'French Republic', 'tail_type': 'loc'}
r4 = {'head': 'Napoleon Bonaparte', 'head_type': 'per', 'type': 'position held', 'tail': 'Emperor', 'tail_type': 'per'}

relations_2 = [r3,r4]

In [ ]:
merge_entities(relations_2)
pretty_print_relations(relations_2)

Similarity:  0.9300509095191956
Relation:
    head: Napoleon
    head_type: per
    type: applies to jurisdiction
    tail: French Republic
    tail_type: loc
Relation:
    head: Napoleon
    head_type: per
    type: position held
    tail: Emperor
    tail_type: per
